# Gradient Boost Classifier for Project Proposal

## Import Necessary Libraries

In [42]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.feature_selection import SelectFromModel
from sklearn.tree import plot_tree
import joblib
import random

### Read Data from .csv File

In [43]:
card_data = pd.read_csv("card_transdata.csv")

### Display All Data

In [44]:
card_data.describe(include="all")

,distance_from_home,distance_from_last_transaction,ratio_to_median_purchase_price,repeat_retailer,used_chip,used_pin_number,online_order,fraud
count,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000
mean,26.628792,5.036519,1.824182,0.881536,0.350399,0.100608,0.650552,0.087403
std,65.390784,25.843093,2.799589,0.323157,0.477095,0.300809,0.476796,0.282425
min,0.004874,0.000118,0.004399,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.878008,0.296671,0.475673,1.000000,0.000000,0.000000,0.000000,0.000000
50%,9.967760,0.998650,0.997717,1.000000,0.000000,0.000000,1.000000,0.000000
75%,25.743985,3.355748,2.096370,1.000000,1.000000,0.000000,1.000000,0.000000
max,10632.723672,11851.104565,267.802942,1.000000,1.000000,1.000000,1.000000,1.000000


### Drop All NaN / NA Values from Dataset

In [45]:
card_data.dropna()

,distance_from_home,distance_from_last_transaction,ratio_to_median_purchase_price,repeat_retailer,used_chip,used_pin_number,online_order,fraud
0,57.877857,0.311140,1.945940,1.0,1.0,0.0,0.0,0.0
1,10.829943,0.175592,1.294219,1.0,0.0,0.0,0.0,0.0
2,5.091079,0.805153,0.427715,1.0,0.0,0.0,1.0,0.0
3,2.247564,5.600044,0.362663,1.0,1.0,0.0,1.0,0.0
4,44.190936,0.566486,2.222767,1.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...
999995,2.207101,0.112651,1.626798,1.0,1.0,0.0,0.0,0.0
999996,19.872726,2.683904,2.778303,1.0,1.0,0.0,0.0,0.0
999997,2.914857,1.472687,0.218075,1.0,1.0,0.0,1.0,0.0
999998,4.258729,0.242023,0.475822,1.0,0.0,0.0,1.0,0.0


### Get Unique Values From the Dependent Vairable Column (Whether the Transction is Fraulent or Not)

In [46]:
print(np.unique(card_data['fraud']))

[0. 1.]


### Now that we know they are unique values of 0 - false or 1 - true

- Convert all floats to ints

In [47]:
card_data['fraud'] = card_data['fraud'].astype(int)

### Radomize the Samples to Drop

In [48]:
random.seed(42)
rows_to_drop = random.sample(range(len(card_data)), 750000)

In [49]:
trimmed_card_data = card_data.drop(rows_to_drop)

In [50]:
trimmed_card_data.describe()

,distance_from_home,distance_from_last_transaction,ratio_to_median_purchase_price,repeat_retailer,used_chip,used_pin_number,online_order,fraud
count,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000
mean,26.596648,5.031841,1.826702,0.881140,0.349708,0.100856,0.649292,0.086808
std,68.246950,33.697220,2.824662,0.323624,0.476879,0.301139,0.477193,0.281554
min,0.004874,0.000118,0.004399,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.864177,0.296690,0.478760,1.000000,0.000000,0.000000,0.000000,0.000000
50%,9.978866,0.998117,1.000741,1.000000,0.000000,0.000000,1.000000,0.000000
75%,25.736275,3.338968,2.102473,1.000000,1.000000,0.000000,1.000000,0.000000
max,10632.723672,11851.104565,267.802942,1.000000,1.000000,1.000000,1.000000,1.000000


In [51]:
print(trimmed_card_data['fraud'].value_counts())

fraud
0    228298
1     21702
Name: count, dtype: int64


## The Following 3 Cells are Commented out when Using Stratified Sampling

In [52]:
minimum_samples = min(trimmed_card_data['fraud'].value_counts())

In [53]:
fraud_samples = trimmed_card_data[trimmed_card_data['fraud'] == 1].sample(minimum_samples,random_state=42)
non_fraud_samples = trimmed_card_data[trimmed_card_data['fraud'] == 0].sample(minimum_samples,random_state=42)

In [54]:
balanced_card_data = pd.concat([fraud_samples,non_fraud_samples])

### Create the Trainning, Testing, and Validation Sets

Choose any 'X' Independent Variable we want to use for the prediction

In [55]:
X = balanced_card_data[['distance_from_home','distance_from_last_transaction','ratio_to_median_purchase_price','repeat_retailer','used_chip','used_pin_number','online_order']]
y = balanced_card_data['fraud']

In [56]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp,y_temp, test_size=0.5, random_state=42)

### Create the Gradient Boosted Decision Trees Classifier

In [57]:
fraud_check_classifier = GradientBoostingClassifier(random_state=42)

## Use Grid Search For Hyper-Parameter Tuning

In [58]:
# Can uncomment top section to take, Grid Search takes awhiled
#Tested Other Features earlier here's a full-scope of what was tested (Had to run through most times to see what was the most reoccuring)...
'''param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [1, 2, 4],
    'max_features': [2, 4, 7],
    'max_leaf_nodes':[3, 5, 8],
    'min_samples_leaf':[1, 3, 5],
    'min_samples_split':[1, 2, 4],
    'learning_rate': [0.05, 0.1, 0.2],
    'loss': ['log_loss']
}


# Initialize the GridSearchCV object
grid_search = GridSearchCV(estimator=fraud_check_classifier, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=3)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Print the best parameters and the best score achieved during the grid search
print("Best parameters:", grid_search.best_params_)
print("Best cross-validation score:", grid_search.best_score_)'''

'param_grid = {\n    \'n_estimators\': [50, 100, 200],\n    \'max_depth\': [1, 2, 4],\n    \'max_features\': [2, 4, 7],\n    \'max_leaf_nodes\':[3, 5, 8],\n    \'min_samples_leaf\':[1, 3, 5],\n    \'min_samples_split\':[1, 2, 4],\n    \'learning_rate\': [0.05, 0.1, 0.2],\n    \'loss\': [\'log_loss\']\n}\n\n\n# Initialize the GridSearchCV object\ngrid_search = GridSearchCV(estimator=fraud_check_classifier, param_grid=param_grid, cv=5, scoring=\'accuracy\', n_jobs=-1, verbose=3)\n\n# Fit the grid search to the data\ngrid_search.fit(X_train, y_train)\n\n# Print the best parameters and the best score achieved during the grid search\nprint("Best parameters:", grid_search.best_params_)\nprint("Best cross-validation score:", grid_search.best_score_)'

### Fit the Model on the Trainning Set

#### Predict the Results

In [59]:
fraud_check_classifier.fit(X_train,y_train)

GradientBoostingClassifier(random_state=42)

### Re-fine through Feature Selection

In [60]:
fraud_check_tree = fraud_check_classifier.estimators_[0, 0]
feature_names = X_train.columns.tolist()
class_names = [str(cls) for cls in sorted(y_train.unique())]
plt.figure(figsize=(200,100), dpi=300)
plot_tree(fraud_check_tree, filled=True, feature_names=feature_names, class_names=class_names, rounded=True)
plt.title('Preliminary Decision Tree from Gradient Boosted Tree')
plt.savefig("./images/gbt_init_bal_tree.svg", format="svg", bbox_inches="tight")
plt.close()

In [61]:
feature_model = SelectFromModel(fraud_check_classifier, prefit=True)
# new x_train and y_train_data
X_train_new_feature = feature_model.transform(X_train)
X_test_new_feature = feature_model.transform(X_test)
X_val_new_feature = feature_model.transform(X_val)

/home/kyle/.local/lib/python3.11/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/home/kyle/.local/lib/python3.11/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/home/kyle/.local/lib/python3.11/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


#### Make New Classifier

In [62]:
fraud_check_classifier_new = GradientBoostingClassifier(
    learning_rate = 0.2,
    n_estimators = 200,
    max_depth = 4,
    max_features = 2,
    max_leaf_nodes = 5,
    min_samples_leaf = 5,
    min_samples_split= 2,
    random_state = 42,
    loss = 'log_loss'
)

fraud_check_classifier_new.fit(X_train_new_feature,y_train)

GradientBoostingClassifier(learning_rate=0.2, max_depth=4, max_features=2,
                           max_leaf_nodes=5, min_samples_leaf=5,
                           n_estimators=200, random_state=42)

In [63]:
y_val_pred = fraud_check_classifier_new.predict(X_val_new_feature)

## Validation Random Forest Graph

In [64]:
fraud_check_tree = fraud_check_classifier_new.estimators_[0, 0]
class_names = [str(cls) for cls in sorted(np.unique(y_val_pred))]
plt.figure(figsize=(200,100), dpi=300)
plot_tree(fraud_check_tree, filled=True, feature_names=X_val_new_feature, class_names=class_names, rounded=True)
plt.title('Validation Tree from Gradient Boosted Classifier')
plt.savefig("./images/validation_bal_tree_gbt.svg", format="svg", bbox_inches="tight")
plt.close()

In [65]:
val_conf_matrix = confusion_matrix(y_val, y_val_pred)
val_class_report = classification_report(y_val, y_val_pred)
print('Validation Confusion Matrix:\n', val_conf_matrix)
print('Validation Classification Report:\n', val_class_report)

Validation Confusion Matrix:
 [[3028  219]
 [ 165 3099]]
Validation Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.93      0.94      3247
           1       0.93      0.95      0.94      3264

    accuracy                           0.94      6511
   macro avg       0.94      0.94      0.94      6511
weighted avg       0.94      0.94      0.94      6511



### Plot the Validation Confusion Matrix

In [67]:
plt.figure(figsize=(8, 6))
sns.heatmap(val_conf_matrix, annot=True, fmt='.2f', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Validation Confusion Matrix (Stratified Sampling)')
plt.savefig("./images/val_bal_gbt_matrix.svg")
plt.close()

### Plot the Validation Classification Report

In [68]:
val_class_report = classification_report(y_val, y_val_pred, target_names=class_names, output_dict=True)
df_val_report = pd.DataFrame(val_class_report).transpose()

plt.figure(figsize=(10, 5))
sns.heatmap(df_val_report.iloc[:-1, :].drop(columns=['support']), annot=True, cmap='Blues', cbar=False)
plt.title('Validation Set: Classification Report (Stratified Sampling)')
plt.savefig("./images/val_bal_gbt_report.svg")
plt.close()

### Plot the Validation Set's Accuracy Score

In [69]:
print("Validation Set Accuracy:", accuracy_score(y_val, y_val_pred))

Validation Set Accuracy: 0.9410228843495623


#### Combine the Validation Set into the Final Trainning Set and Re-Run

In [70]:
X_final_training_set = np.vstack([X_train_new_feature, X_val_new_feature])
y_final_training_set = pd.concat([y_train, y_val])

In [71]:
fraud_check_classifier_final = GradientBoostingClassifier(
    learning_rate = 0.2,
    n_estimators = 200,
    max_depth = 4,
    max_features = 2,
    max_leaf_nodes = 5,
    min_samples_leaf = 5,
    min_samples_split= 2,
    random_state = 42,
    loss = 'log_loss'
)

fraud_check_classifier_final.fit(X_final_training_set, y_final_training_set)

# Final evaluation on the test set
y_test_pred = fraud_check_classifier_final.predict(X_test_new_feature)

In [72]:
fraud_check_tree = fraud_check_classifier_final.estimators_[0, 0]
class_names = [str(cls) for cls in sorted(y_final_training_set.unique())]
plt.figure(figsize=(200,100))
plot_tree(fraud_check_tree, filled=True, feature_names=X_final_training_set, class_names=class_names, rounded=True)
plt.title('Final Tree from Gradient Boosted Tree Classifier')
plt.savefig("./images/final_bal_tree_gbt.svg", format="svg", bbox_inches="tight")
plt.close()

In [73]:
conf_matrix = confusion_matrix(y_test, y_test_pred)
class_report = classification_report(y_test, y_test_pred, target_names=class_names)
print('Confusion Matrix:\n', conf_matrix)
print('Classification Report:\n', class_report)

Confusion Matrix:
 [[2962  242]
 [ 181 3126]]
Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.92      0.93      3204
           1       0.93      0.95      0.94      3307

    accuracy                           0.94      6511
   macro avg       0.94      0.93      0.93      6511
weighted avg       0.94      0.94      0.94      6511



### Plot confusion matrix

In [74]:
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='.2f', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.savefig("./images/final_bal_conf_matrix_gbt.svg", format="svg", bbox_inches="tight")
plt.close()

### Plot the Final Classification Report 

In [75]:
class_report = classification_report(y_test, y_test_pred, target_names=class_names, output_dict=True)
df_final_report = pd.DataFrame(class_report).transpose()

plt.figure(figsize=(10, 5))
sns.heatmap(df_final_report.iloc[:-1, :].drop(columns=['support']), annot=True, cmap='Blues', cbar=False)
plt.title('Classification Report (Stratified Sampling)')
plt.savefig("./images/final_bal_class_report_gbt.svg", format="svg", bbox_inches="tight")
plt.close()

### Plot the Final Accuracy Score

In [76]:
print("Accuracy:", accuracy_score(y_test, y_test_pred))


Accuracy: 0.9350330210413147


### Saving the Created Model for 'Live' Testing

In [77]:
joblib.dump(fraud_check_classifier_final, "./models/fraud_checker_bal_Gradient_Boost.pkl")

['./models/fraud_checker_bal_Gradient_Boost.pkl']

### Conclusion

Applying the Gradient Classifier with the multiple independent variables to predict the dependent variable (whether the transaction is fraudlent or not), yielded significant results based on the statistics provided after checking for model accuracy utilizing tuning of hyper-parameters with grid-search and cross-validaiton, and optimal feature selection of the data set.